# Chicago Taxi Data Analysis (2023)

## Dataset Overview

This analysis focuses on taxi trips in Chicago during the year 2023, using data reported to the City of Chicago in its role as a regulatory agency. The dataset includes detailed trip information such as start and end times, trip distances, fares, and payment methods.

### Privacy Considerations
- **Taxi ID**: A consistent identifier is used for each taxi medallion number, but the actual medallion number is not shown to protect privacy.
- **Census Tracts**: Location details such as pickup and drop-off Census Tracts are suppressed in some cases.
- **Timestamp Rounding**: Trip start and end times are rounded to the nearest 15 minutes.

### Limitations
Not all trips are reported due to the data collection process; however, the City of Chicago believes the dataset represents the majority of trips.

For more details about the dataset and its creation, see [this page](https://console.cloud.google.com/marketplace/product/city-of-chicago-public-data/chicago-taxi-trips?hl=en&organizationId=0&project=velvety-outcome-444601-r0) on the City of Chicago's blog.

# 1. Load Data

To start the analysis, we load the dataset from Google Drive where the monthly taxi trip CSV files are stored. The Google Drive plugin is used to access and download the data directly into the Jupyter environment.

### Steps:
1. Authenticate access to Google Drive.
2. Download the monthly CSV files into the working directory.
3. Load the CSV files into a Pandas DataFrame for processing.

In [ ]:
from google.colab import drive
import pandas as pd
import os
import glob

# Mount Google Drive
drive.mount('/content/drive')

# Path to the folder containing the files
folder_path = '/content/drive/My Drive/Chicago Taxi Data/'

# List all CSV files in the folder
file_paths = glob.glob(os.path.join(folder_path, '*.csv'))

# Load each file into a DataFrame and combine them
dataframes = []
for file_path in file_paths:
    print(f"Loading data from {file_path}...")
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Combine all months into one DataFrame
all_data = pd.concat(dataframes, ignore_index=True)

# Display basic information about the combined data
print("Data loaded successfully. Preview:")
print(all_data.head())

Mounted at /content/drive
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-jan2023.csv...
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-feb2023.csv...
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-mar2023.csv...
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-apr2023.csv...
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-may2023.csv...
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-jun2023.csv...
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-jul2023.csv...


<ipython-input-1-59bd3cead216>:19: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-aug2023.csv...


<ipython-input-1-59bd3cead216>:19: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-sep2023.csv...
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-oct2023.csv...
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-nov2023.csv...
Loading data from /content/drive/My Drive/Chicago Taxi Data/chicago-taxi-dec2023.csv...
Data loaded successfully. Preview:
                                 unique_key  \
0  ea138d778cb63034cabcbb81031b7946a513e08b   
1  008901346b4899be1caacc775e605d0b01ce6ea9   
2  063f013869c8293a083c6b9d0f446055e539a9a5   
3  11cec01787dab2f4c875356b74eecbaae6a06431   
4  29767d8c282e30f4c1c23fe0a8f14424d4567918   

                                             taxi_id     trip_start_timestamp  \
0  82436a5c2f9503cc461215c839f24fa16fa7f02d9390bb...  2023-01-31 10:45:00 UTC   
1  ed17822a864c82f0ac8c3110918dacfa804f66b06d9668...  2023-01-18 19:45:00 UTC   
2  7ed122481c0964a5555309bf4696e25bbf7def086d7ecb...  2023-01-27 22:00:00 UTC   
3  98a

## 2) ETL (Extract, Transform, Load) and Data Segmentation

We now perform data cleaning, transformations, and feature engineering on different segments of the data. Each segmentation targets a specific analytical goal. By creating separate, cleaned DataFrames for each segmentation, we ensure transformations don’t interfere with one another.

## 2.1 Revenue and Profitability Analysis
The goal of this segmentation is to understand the relationship between trip distance, time, and revenue. By calculating metrics like revenue per mile and revenue per minute, we can identify high-efficiency trips and explore profitability patterns. Additionally, tip percentages provide insights into customer tipping behavior.


In [ ]:
# Create a copy of the data for revenue and profitability analysis
revenue_data = all_data.copy()

# Remove rows with missing or zero values in critical columns
revenue_data = revenue_data.dropna(subset=['trip_total', 'trip_miles', 'trip_seconds'])
revenue_data = revenue_data[(revenue_data['trip_miles'] > 0) & (revenue_data['trip_total'] > 0)]

# Remove outliers (e.g., trips with extremely high fares or distances)
revenue_data = revenue_data[(revenue_data['trip_miles'] < 100) & (revenue_data['trip_total'] < 1000)]

# Add new metrics
revenue_data['revenue_per_mile'] = revenue_data['trip_total'] / revenue_data['trip_miles']
revenue_data['revenue_per_minute'] = revenue_data['trip_total'] / (revenue_data['trip_seconds'] / 60)
revenue_data['tip_percentage'] = (revenue_data['tips'] / revenue_data['fare']) * 100


<ipython-input-2-e1610ec035be>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenue_data['revenue_per_mile'] = revenue_data['trip_total'] / revenue_data['trip_miles']
<ipython-input-2-e1610ec035be>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revenue_data['revenue_per_minute'] = revenue_data['trip_total'] / (revenue_data['trip_seconds'] / 60)
<ipython-input-2-e1610ec035be>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

## 2.2 Demand Patterns by Time and Location
This segmentation focuses on identifying when and where demand is highest. By extracting time-based features (e.g., hour of day, day of week, month), we can uncover temporal trends in trip demand. This information helps highlight peak demand periods and geographic hotspots.


In [ ]:
# Create a copy of the data for demand patterns
demand_data = all_data.copy()

# Remove rows with missing or invalid timestamps
demand_data = demand_data.dropna(subset=['trip_start_timestamp'])
demand_data['trip_start_timestamp'] = pd.to_datetime(demand_data['trip_start_timestamp'])

# Ensure valid latitude and longitude
demand_data = demand_data.dropna(subset=['pickup_latitude', 'pickup_longitude'])
demand_data = demand_data[(demand_data['pickup_latitude'].between(-90, 90)) &
                          (demand_data['pickup_longitude'].between(-180, 180))]

# Add new time-based metrics
demand_data['hour_of_day'] = demand_data['trip_start_timestamp'].dt.hour
demand_data['day_of_week'] = demand_data['trip_start_timestamp'].dt.day_name()
demand_data['month'] = demand_data['trip_start_timestamp'].dt.month_name()


## 2.3 Payment Method Analysis
This segmentation explores how payment methods (e.g., cash vs. card) influence revenue and tipping behavior. By calculating average tips and revenue for each payment type, we can identify customer preferences and analyze how payment methods impact profitability.


In [ ]:
# Create a copy of the data for payment method analysis
payment_data = all_data.copy()

# Remove rows with missing or invalid values
payment_data = payment_data.dropna(subset=['payment_type', 'fare', 'tips'])
payment_data = payment_data[payment_data['fare'] > 0]

# Summarize payment-specific metrics
payment_summary = payment_data.groupby('payment_type').agg({
    'tips': 'mean',
    'trip_total': 'sum',
    'fare': 'mean'
}).reset_index()
payment_summary['avg_tip_percentage'] = (payment_summary['tips'] / payment_summary['fare']) * 100


## 2.4 Geospatial Trip Efficiency
This segmentation examines how trip efficiency varies across different pickup and drop-off locations. Metrics like efficiency ratio (fare per mile) and trip duration in minutes help assess pricing fairness and profitability in different geographic areas.


In [ ]:
# Create a copy of the data for geospatial analysis
geospatial_data = all_data.copy()

# Remove rows with missing or invalid values
geospatial_data = geospatial_data.dropna(subset=['trip_miles', 'fare', 'pickup_latitude', 'pickup_longitude'])
geospatial_data = geospatial_data[(geospatial_data['trip_miles'] > 0) &
                                  (geospatial_data['pickup_latitude'].between(-90, 90)) &
                                  (geospatial_data['pickup_longitude'].between(-180, 180))]

# Add geospatial metrics
geospatial_data['efficiency_ratio'] = geospatial_data['fare'] / geospatial_data['trip_miles']
geospatial_data['trip_duration_minutes'] = geospatial_data['trip_seconds'] / 60


## 2.5 Temporal Revenue Trends
This segmentation focuses on analyzing revenue patterns over time. By aggregating daily and hourly revenues, we can identify key trends and predict high-revenue periods. This is useful for understanding seasonal changes and operational planning.


In [ ]:
# Create a copy of the data for temporal analysis
temporal_data = all_data.copy()

# Remove rows with missing or invalid values
temporal_data = temporal_data.dropna(subset=['trip_start_timestamp', 'trip_total'])
temporal_data['trip_start_timestamp'] = pd.to_datetime(temporal_data['trip_start_timestamp'])

# Aggregate daily and hourly revenue
temporal_data['trip_date'] = temporal_data['trip_start_timestamp'].dt.date
daily_revenue = temporal_data.groupby('trip_date')['trip_total'].sum().reset_index()
hourly_revenue = temporal_data.groupby(temporal_data['trip_start_timestamp'].dt.hour)['trip_total'].sum().reset_index()


## 3) Download Prepared Data

After performing ETL and segmentation, we export the cleaned and enriched datasets for use in Tableau or other visualization tools.


In [ ]:
# Export each segmentation
revenue_data.to_csv('/content/drive/My Drive/Revenue_Data.csv', index=False)
demand_data.to_csv('/content/drive/My Drive/Demand_Data.csv', index=False)
payment_summary.to_csv('/content/drive/My Drive/Payment_Summary.csv', index=False)
geospatial_data.to_csv('/content/drive/My Drive/Geospatial_Data.csv', index=False)
daily_revenue.to_csv('/content/drive/My Drive/Daily_Revenue.csv', index=False)
hourly_revenue.to_csv('/content/drive/My Drive/Hourly_Revenue.csv', index=False)
